# 최근 언어모델 기반 평가지표

## BertScore
- 개발 목적  
    기존의 n-gram 기반 평가 지표(BLEU, ROUGE 등)의 한계를 극복하기 위해 개발. 이전 지표들은 단어의 표면적 일치에 초점을 맞추어 의미적 유사성 파악 부족. BERTScore는 문맥화된 임베딩을 활용하여 의미적 유사성을 더 정확하게 측정하기 위해
- 핵심 개념 : "의미적 유사성"  
    문맥화된 임베딩을 활용하여 의미적 유사성을 더 정확하게 측정

- 한계
    - 계산 복잡성: BLEU나 ROUGE보다 계산 비용이 큼
    - 모델 의존성: 사용하는 사전학습 모델에 따라 결과가 달라질 수 있어 일관된 비교가 어려움
    - 도메인 특수성: 특정 도메인에 특화된 텍스트를 평가 어려움

- 발전
    - 다양한 사전학습 모델과의 통합: RoBERTa, DeBERTa, XLM-RoBERTa 등
    - 세분화된 평가: 문장 단위뿐만 아니라 문서 단위, 문단 단위로 확장
    - 다국어 지원: 영어 중심에서 다양한 언어로
    - 특수 도메인 적응: 의학, 법률 등 특수 도메인 맞춤

- 결론  
    BERTScore는 의미론적 이해를 크게 향상시킨 평가 지표다. 현재 기계 번역, 요약, 대화 시스템 등 다양한 자연어 생성 테스크에서 표준 평가 지표로 자리잡았다. 계산 비용이라는 단점에도 불구하고, 의미 기반 평가의 중요성이 부각되면서 그 활용도는 계속 확대되고 있다.

## 주요 평가지표
- METEOR (Metric for Evaluation of Translation with Explicit ORdering)

    - 강점:

        동의어/어간 매칭: 형태소 분석과 WordNet 기반 동의어 사전을 활용해 의미적 유사성 평가 가능

        패널티 시스템: 단어 삽입/삭제 오류에 가중치 차등 적용

        BLEU 대비 개선: recall 고려와 n-gram 유연성으로 인간 평가와 0.96 상관관계 달성

    - 한계:

        언어 의존성: WordNet 없는 언어에 적용 시 정확도 저하

        계산 복잡도: BLEU 대비 2~3배 긴 처리 시간


- GLUE/SuperGLUE (General Language Understanding Evaluation)

    - 강점:

        11개 NLU 태스크 통합: CoLA(문법 검증), MNLI(추론) 등 다양한 과제 평가

        인간 기준점 제공: 2019년 BERT 모델이 인간 성능 초과

    - 한계:

        생성 능력 미반영: 텍스트 이해 중심 평가로 생성 품질 측정 불가

        데이터 유출 문제: 대규모 사전학습 모델에서 테스트 데이터 노출 가능성

- KLUE:
  
    - 강점  
        포괄적인 평가: KLUE는 8개의 다양한 한국어 자연어 이해 태스크를 포함하여 종합적인 평가가 가능.

        한국어 특화: 세계에서 13번째로 많이 사용되는 한국어에 특화된 벤치마크로, 한국어 NLP 발전에 크게 기여.

        윤리적 고려: 데이터 구축 시 저작권, 개인정보 보호, 사회적 편견 등을 고려.

        오픈 소스: 누구나 자유롭게 사용할 수 있어 한국어 NLP 연구 발전을 촉진.

        다양한 평가 지표: 각 태스크에 적합한 평가 지표를 제공.
    - 한계  
        제한된 도메인: 외국어나 일부 특수한 도메인(예: 금융)에 대한 평가는 제한적.

## BERTScore 실습

In [2]:
# 필요 라이브러리 설치
!pip install bert-score

  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-man

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bert_score import score
from transformers import AutoTokenizer, AutoModel
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
import nltk.data
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# 데이터 예시 생성
references = [
    "이 영화는 매우 재미있고 감동적이었습니다.",
    "서울은 한국의 수도이며 많은 관광 명소가 있습니다.",
    "인공지능 기술은 빠르게 발전하고 있습니다.",
    "건강을 위해 규칙적인 운동과 균형 잡힌 식단이 중요합니다.",
    "환경 보호를 위해 재활용을 실천해야 합니다."
]

candidates = [
    "이 영화는 정말 재미있었고 감동적인 내용이었습니다.",  # 유사한 문장
    "대한민국의 수도 서울에는 관광 명소가 많이 있습니다.",  # 유사한 문장, 단어 순서 변경
    "머신러닝과 딥러닝은 최근 급속도로 발전하고 있는 분야입니다.",  # 의미는 유사하지만 다른 단어 사용
    "건강한 생활을 위해서는 운동과 식이 요법이 중요합니다.",  # 유사한 의미, 다른 표현
    "환경을 보존하기 위한 노력이 필요합니다."  # 관련은 있지만 덜 유사한 문장
]

In [ ]:
# BERTScore 계산 함수
def calculate_bertscore(refs, cands, model_type="bert-base-multilingual-cased", lang="ko"):
    """
    BERTScore를 계산하는 함수

    Parameters:
        refs (list): 참조 문장 리스트
        cands (list): 생성된 문장 리스트
        model_type (str): 사용할 BERT 모델의 종류
        lang (str): 언어 코드

    Returns:
        pandas.DataFrame: 각 문장 쌍의 BERTScore (Precision, Recall, F1)
    """
    # BERTScore 계산
    P, R, F1 = score(cands, refs, lang=lang, model_type=model_type, verbose=True)

    # 결과를 DataFrame으로 변환
    results = pd.DataFrame({
        'Reference': refs,
        'Candidate': cands,
        'Precision': P.numpy(),
        'Recall': R.numpy(),
        'F1': F1.numpy()
    })

    # 평균 점수 계산
    avg_scores = {
        'Average Precision': P.mean().item(),
        'Average Recall': R.mean().item(),
        'Average F1': F1.mean().item()
    }

    return results, avg_scores

In [ ]:
# BLEU 점수 계산 함수 (비교를 위해)
def calculate_bleu(refs, cands):
    scores = []
    for ref, cand in zip(refs, cands):
        # 토큰화
        ref_tokens = nltk.word_tokenize(ref)
        cand_tokens = nltk.word_tokenize(cand)
        # BLEU 계산 (1-gram부터 4-gram까지)
        score = sentence_bleu([ref_tokens], cand_tokens, weights=(0.25, 0.25, 0.25, 0.25))
        scores.append(score)
    return scores

# METEOR 점수 계산 함수 (비교를 위해)
def calculate_meteor(refs, cands):
    scores = []
    for ref, cand in zip(refs, cands):
        # 토큰화
        ref_tokens = nltk.word_tokenize(ref)
        cand_tokens = nltk.word_tokenize(cand)
        # METEOR 계산
        score = meteor_score([ref_tokens], cand_tokens)
        scores.append(score)
    return scores

In [ ]:
# BERTScore 계산
bertscore_results, bertscore_avg = calculate_bertscore(references, candidates)

# 비교를 위한 다른 평가 지표 계산
bleu_scores = calculate_bleu(references, candidates)
meteor_scores = calculate_meteor(references, candidates)

# 결과 출력
print("\n=== BERTScore 결과 ===")
print(bertscore_results)
print("\n=== 평균 BERTScore ===")
for k, v in bertscore_avg.items():
    print(f"{k}: {v:.4f}")

print("\n=== 다른 평가 지표와 비교 ===")
comparison_df = pd.DataFrame({
    'Reference': references,
    'Candidate': candidates,
    'BERTScore (F1)': bertscore_results['F1'],
    'BLEU': bleu_scores,
    'METEOR': meteor_scores
})
print(comparison_df)

In [ ]:
# 추가 실험: 다양한 품질의 문장 생성 결과 비교
print("\n=== 다양한 품질의 문장 생성 결과 비교 ===")

reference = "인공지능 기술은 우리의 일상생활을 크게 변화시키고 있습니다."
candidates_quality = [
    "AI 기술로 인해 우리의 일상생활이 크게 바뀌고 있습니다.",  # 좋은 품질
    "인공지능은 생활 방식을 바꾸고 있습니다.",  # 중간 품질 (일부 정보 누락)
    "기술이 발전하고 있습니다.",  # 낮은 품질 (많은 정보 누락)
    "오늘 날씨는 맑고 화창합니다."  # 관련 없는 문장
]

# 각 후보 문장에 대한 BERTScore 계산
quality_results = []
for idx, candidate in enumerate(candidates_quality):
    P, R, F1 = score([candidate], [reference], lang="ko", model_type="bert-base-multilingual-cased")
    quality_results.append({
        'Quality': ['Good', 'Medium', 'Poor', 'Unrelated'][idx],
        'Candidate': candidate,
        'BERTScore F1': F1.item()
    })

quality_df = pd.DataFrame(quality_results)
print(quality_df)


=== 다양한 품질의 문장 생성 결과 비교 ===
     Quality                         Candidate  BERTScore F1
0       Good  AI 기술로 인해 우리의 일상생활이 크게 바뀌고 있습니다.      0.881312
1     Medium            인공지능은 생활 방식을 바꾸고 있습니다.      0.873494
2       Poor                    기술이 발전하고 있습니다.      0.811601
3  Unrelated                  오늘 날씨는 맑고 화창합니다.      0.704634


In [ ]:
# 모델별 BERTScore 비교 (한국어 모델)
print("\n=== 다양한 사전학습 모델별 BERTScore 비교 ===")
models = ["bert-base-multilingual-cased"]
model_results = []

for model in models:
    P, R, F1 = score(candidates[:3], references[:3], lang="ko", model_type=model)
    model_results.append({
        'Model': model,
        'Average F1': F1.mean().item()
    })

model_df = pd.DataFrame(model_results)
print(model_df)


=== 다양한 사전학습 모델별 BERTScore 비교 ===
                          Model  Average F1
0  bert-base-multilingual-cased    0.873183


In [ ]:
# 사용자 정의 문장 평가를 위한 인터페이스
from IPython.display import display
from ipywidgets import widgets

def evaluate_custom_text(reference, candidate):
    P, R, F1 = score([candidate], [reference], lang="ko", model_type="bert-base-multilingual-cased")
    print(f"Reference: {reference}")
    print(f"Candidate: {candidate}")
    print(f"BERTScore - Precision: {P.item():.4f}, Recall: {R.item():.4f}, F1: {F1.item():.4f}")

# 위젯 생성
reference_widget = widgets.Textarea(
    value='인공지능 기술은 우리의 일상생활을 크게 변화시키고 있습니다.',
    description='Reference:',
    layout=widgets.Layout(width='80%', height='80px')
)

candidate_widget = widgets.Textarea(
    value='AI 기술로 인해 우리의 일상생활이 크게 바뀌고 있습니다.',
    description='Candidate:',
    layout=widgets.Layout(width='80%', height='80px')
)

button = widgets.Button(description="Evaluate")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        evaluate_custom_text(reference_widget.value, candidate_widget.value)

button.on_click(on_button_clicked)

# 위젯 표시
print("\n=== 사용자 정의 문장 평가 ===")
print("아래 텍스트 상자에 문장을 입력하고 Evaluate 버튼을 클릭하세요.")
display(reference_widget)
display(candidate_widget)
display(button)
display(output)



=== 사용자 정의 문장 평가 ===
아래 텍스트 상자에 문장을 입력하고 Evaluate 버튼을 클릭하세요.


Textarea(value='인공지능 기술은 우리의 일상생활을 크게 변화시키고 있습니다.', description='Reference:', layout=Layout(height='80px', wid…

Textarea(value='AI 기술로 인해 우리의 일상생활이 크게 바뀌고 있습니다.', description='Candidate:', layout=Layout(height='80px', widt…

Button(description='Evaluate', style=ButtonStyle())

Output()


=== BERTScore의 장점 ===
1. 의미적 유사성 포착: 단어 순서와 상관없이 의미적 유사성을 측정
2. 문맥 고려: BERT의 문맥 이해 능력을 활용하여 더 정확한 평가
3. 다국어 지원: 다양한 언어에 적용 가능
4. 유연성: 다양한 사전학습 모델을 사용할 수 있음

=== BERTScore의 한계 ===
1. 계산 비용: BLEU나 METEOR보다 계산 비용이 큼
2. 모델 의존성: 사용하는 사전학습 모델에 따라 결과가 달라질 수 있음
3. 도메인 제한: 특정 도메인에 맞게 모델을 선택해야 할 수 있음


# 결론
### === BERTScore의 장점 ===
1. 의미적 유사성 포착: 단어 순서와 상관없이 의미적 유사성을 측정"
1. 문맥 고려: BERT의 문맥 이해 능력을 활용하여 더 정확한 평가"
1. 다국어 지원: 다양한 언어에 적용 가능"
1. 유연성: 다양한 사전학습 모델을 사용할 수 있음"

### === BERTScore의 한계 ==="
1. 계산 비용: BLEU나 METEOR보다 계산 비용이 큼"
1. 모델 의존성: 사용하는 사전학습 모델에 따라 결과가 달라질 수 있음"
1. 도메인 제한: 특정 도메인에 맞게 모델을 선택해야 할 수 있음"